In [302]:
from bs4 import BeautifulSoup

In [303]:
html = '''
<html>
<table border=1>
    <tr>
        <td> 항목 </td>
        <td> 2013 </td>
        <td> 2014 </td>
        <td> 2015 </td>
    </tr>

    <tr>
        <td> 매출액 </td>
        <td> 100 </td>
        <td> 200 </td>
        <td> 300 </td>
    </tr>
</table>
</html>
'''

In [304]:
soup = BeautifulSoup(html, 'html5lib')
result = soup.select('td:nth-of-type(1)')
print(result)

[<td> 항목 </td>, <td> 매출액 </td>]


In [305]:
html = '''
<ul>
    <li> 100 </li> 
    <li> 200 </li>
</ul> 
<ol>
    <li> 300 </li> 
    <li> 400 </li>
</ol>
'''
soup = BeautifulSoup(html, 'html5lib') 
result = soup.select('ol li:nth-of-type(2)')
for r in result: 
    print(r.text)

 400 


In [306]:
import requests
import re
response = requests.get("http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=035720") #끝에 cmp_cd : 종목코드
print(response.text)



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ko">
<head>
	<title>온라인기업정보 - 기업모니터 - 기업개요(카카오)</title>
	<link rel="shortcut icon" href="/favicon.ico" />
	<meta http-equiv="X-UA-Compatible" content="IE=edge" />
	<script language="javascript" src="/include/domain.js" type="text/javascript"></script>
	<!--[if (!IE) | (gt IE 8)]>
    	<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />
    <![endif]-->
	<!--[if IE 6]>
            <link href="/include/only_ie6.css" type="text/css" rel="stylesheet"/>
        <![endif]-->
	<!--[if lt IE 8]>
            <link href="/include/lt_ie8.css" type="text/css" rel="stylesheet"/>
        <![endif]-->
	<link href="/v1/include/css?v=q7Pq73b9jj00gmFmOVUSxzXv_5FtLNW9qxkqFwJqh-g1" rel="stylesheet"/>

	<link href="/v1/include/css1?v=2Kc54WAY0-YKwI0WYHBphRe4FmbyttZ0_0A_hAhAjHc1" rel="stylesheet"/>

    <link href="/v1/include/csscrawler?v=

In [89]:
code = "035720"
#compile(패턴, 플래그) 만약 패턴과 플래그가 동일한 정규식을 여러번 사용하려면 compile()를 사용하여 지정한 다음, 위의 함수들을 사용할 수 있습니다. / re.IGNORECASE : 대소문자 구분X
re_enc = re.compile("encparam: '(.*)'", re.IGNORECASE)              
re_id = re.compile("id: '([a-zA-Z0-9]*)' ?", re.IGNORECASE)

url = f"http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd={code}"
html = requests.get(url).text
encparam = re_enc.search(html).group(1) 
encid = re_id.search(html).group(1)

url = f"http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd={code}&fin_typ=0&freq_typ=A&encparam={encparam}&id={encid}"
headers = {"Referer": "HACK"}
html = requests.get(url, headers=headers).text 
print(html)

<table class="hTDdCYnFkT0 gHead01 all-width" summary="주요재무정보를 제공합니다."><caption class="blind">"주요재무정보"</caption><thead><tr><th>7271</th><th>7271</th><th>7271</th><th>7271</th><th>7271</th><th>7271</th><th>7271</th><th>7271</th><th>7271</th></tr></thead><tbody><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td></tr><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td></tr><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td></tr><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td></tr><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td></tr><tr><td>7,271</td><td>7,271</td><td>7,271</td><td>7,271</td><td>7,

In [308]:
import pandas as pd

dfs = pd.read_html(html)

ValueError: No tables found

In [100]:
df = dfs[1]['연간연간컨센서스보기']

In [114]:
df.index = dfs[1]['주요재무정보'].values.flatten()

In [117]:
df.loc['현금배당수익률']

2019/12  (IFRS연결)       0.08
2020/12  (IFRS연결)       0.04
2021/12  (IFRS연결)       0.05
2022/12(E)  (IFRS연결)    0.07
Name: 현금배당수익률, dtype: float64

---
# 논문 크롤링 시도

In [309]:
from bs4 import BeautifulSoup as bs
import urllib.request
import re
from selenium import webdriver
import pandas as pd
import datetime
import os
import getpass

In [310]:
def get_URL(page):
    url_before_page = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&queryText=&iStartCount="
    url_after_page = "orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0"
    # http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0
    # http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=10&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0
    URL = url_before_page + page + url_after_page
    return URL

In [312]:
get_URL(str(10))

'http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&queryText=&iStartCount=10orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0'

In [313]:
URL = get_URL(str(0))
source_code = urllib.request.urlopen(URL)
soup = bs(source_code, 'lxml', from_encoding='utf-8')
print(soup)

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no-referrer-when-downgrade" name="referrer"/>
<meta content="width=device-width" name="viewport"/>
<meta content="telephone=no" name="format-detection"/><!-- 아이폰 전화번호 스타일 초기화 -->
<meta content="Copyright(c) KERIS. ALL RIGHTS  RESERVED" name="copyright"/>
<title>RISS 검색 - 국내학술지논문</title>
<link href="/commons/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/commons/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/commons/css/base.css?20220324" rel="stylesheet" type="text/css"/>
<link href="/commons/css/common.css?20220324" rel="stylesheet" type="text/css"/>
<link href="/search/css/search.css?20220324" rel="stylesheet" type="text/css"/>
<link href="/commons/css/jquery.mCustomScrollbar.css?20220324" rel="stylesheet" type="text/css"/>
<link href="/commons/css/relation/dtree.css?20220324" rel="StyleSheet" type="text/c

In [182]:
soup.select('li > div.cont > p.title > a')[0]['href']

'/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f77ebda6182557e3b7998d826d417196&keyword=빅데이터 빅데이터'

In [183]:
def get_link(csv_name, page_num):
    
    for i in range(page_num):
        current_page = i*10
        URL = get_URL(str(current_page))
        source_code_from_URL = urllib.request.urlopen(URL)
        soup = bs(source_code_from_URL, 'lxml', from_encoding='utf-8')  #lxml이 더 빠름
        
        for j in range(10):
            paper_link = soup.select('li > div.cont > p.title > a')[j]['href']
            paper_url = 'http://riss.or.kr' + paper_link
            
            reference_data = get_reference(paper_url)
            
            save_csv(csv_name, reference_data)

In [202]:
webdriver.ChromeOptions().binary_location

''

In [360]:
from selenium import webdriver
from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
browser = webdriver.Chrome(r'C:/Users/DJK-Office/Desktop/webdriver/msedgedriver.exe', options=options)  #edge driver
webdriver
browser.get(URL)        
html = browser.page_source     #page source를 bs4로 가져오기
soup = bs(html, "html.parser")
# print(soup)
# title = soup.find("h3", "title")

from selenium import webdriver
import bs4
import time
from selenium.webdriver.common.keys import Keys

#키워드 검색
driver = webdriver.Chrome(r'C:/Users/DJK-Office/Desktop/webdriver/msedgedriver.exe', options=options) #webdriver 위치 (자기의 컴퓨터 위치에 맞춰 지정)
keyword = input() 
xpath = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery='+keyword+'&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=' + keyword 
driver.get(xpath)

answer = [] #크롤링한 정보가 들어갈 리스트

check_first_page = 1 #첫 번째 페이지(1-10)에서 논문이 끝난 경우를 탐지하는 변수
for_check = 1 # 다음 페이지가 존재하지 않는 경우를 탐지하는 변수 

for j in range(3,13): #3~12 다음 페이지까지 (1~10)
    try: 
    #첫 페이지에 있는 10개의 논문 크롤링 시작
        for i in range(1,11):
            move = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[2]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
            driver.find_element_by_xpath(move).send_keys(Keys.CONTROL + '\n') #새 탭 열기
            driver.switch_to.window(driver.window_handles[1])

            plus = [] #논문에 넣을 정보들
            element = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[1]/h3')
            plus.append(element.text) #이름
            element2 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[1]/div/p/a')
            plus.append(element2.text) #저자
            element3 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[4]/div/p')
            plus.append(element3.text) #년도

            for k in range(1,3): #초록
                try:
                    element4 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(k)+']/div[2]/p')
                except: #초록이 없으면 그냥 지나간다
                    continue
                plus.append(element4.text)       
            answer.append(plus)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
    except:
        check_first_page = 0 
        break
        
    #한 페이지 논문 다 크롤링하면 다음 페이지로
    page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/a['+str(j)+']' 
    driver.find_element_by_xpath(page).click();    

#page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/a[12]' 
#driver.find_element_by_xpath(page).click();    


#(1~10) 이랑 (11~20)부터 다음 페이지로 이동하는 크롤링 키가 다르므로 반복문을 따로 작성 -- (11~20)부터 크롤링하는 코드
if(check_first_page !=0):
    while(True):
        if(for_check !=1): #페이지가 없다는 오류 메시지를 받으면 크롤링을 멈추기
            break
        for j in range(4,14): #11~20page + 다음 페이지 누르기 --> for 문 돌리면 (21-30)으로
            try: 
                #각 페이지당 크롤링 시작(반복문 시작 시 11page니 크롤링 먼저 하고 다음 페이지로 이동)
                for i in range(1,11):
                    move = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[2]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
                    driver.find_element_by_xpath(move).send_keys(Keys.CONTROL + '\n') #새 탭 열기
                    driver.switch_to.window(driver.window_handles[1])

                    plus = [] #논문에 넣을 정보들
                    element = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[1]/h3')
                    plus.append(element.text) #이름
                    element2 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[1]/div/p/a')
                    plus.append(element2.text) #저자
                    element3 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[4]/div/p')
                    plus.append(element3.text) #년도

                    for k in range(1,3): #초록
                        try:
                            element4 = driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(k)+']/div[2]/p')
                        except: #초록이 없으면 그냥 지나간다
                            continue
                        plus.append(element4.text)
        
                    answer.append(plus)
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                
            except:
                for_check = 0
                break
            page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/a['+str(j)+']'
            driver.find_element_by_xpath(page).click();
            time.sleep(0.5)     

print("크롤링한 논문 개수 :",len(answer),"개\n")
for i in range(len(answer)):
    print(answer[i])

In [337]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
a = urlopen('https://www.naver.com/')
# a를 호출해 html을 parser
soup = bs(a.read(), 'html.parser')
# soup에서 h1 tag 하나를 가져온다.
print(soup.h1)

<h1 class="logo_default">
<a class="logo_naver" data-clk="top.logo" href="/"><span class="blind">네이버</span></a>
</h1>


In [571]:
browser.find_element_by_xpath("//*[@id='divContent']/div/div[2]/div/div[3]/div[2]/ul/li[1]/div[2]/p[1]/a").text


'지도서비스를 이용한 위치 기반 관광 빅데이터의 시각화'

In [575]:
browser.find_element_by_xpath("//*[@id='divContent']/div/div[2]/div/div[3]/div[2]/ul/li[1]/div[2]").text


'지도서비스를 이용한 위치 기반 관광 빅데이터의 시각화\n김경식,김진국,조완섭,류관희,Kyeongsik Kim,Jinkook Kim,Wan-Sup Cho,Kwan-Hee Yoo 한국빅데이터서비스학회 2014 한국빅데이터서비스학회 논문지 Vol.1 No.1\n원문보기\n구독기관에 따라 유료논문이 존재할 수 있습니다.\n디지털 경제의 확산은 무수히 많은 양의 자료와 정보를 양산시켰으며 경제, 사회, 정치 등 다양한 분야에서의 빅데이터의 활용성을 증가시켰다. 사기업 경영활동, 정부, 시민 단체 등 사회 전반적인 분야에서 빅데이터는 주요한 전략적 근거가 되고 있다. 하지만 국내 관광 분야에서의 빅데이터 활 용은 미비한 상태이다. 따라서 관광 정책 또는 사업에서 빅데이터 활용은 전략적 관광 정책에 있어서 경쟁 우위 요소가 될 것이며, 국내 관광 산업의 잠재력을 확대하고 발전시키는 핵심이 될 것 이다. 본 논문에서는 지역 관광 명소의 빅데이터 수집하여 분석하고 더 나아가 분석 결과를 지도 를 활용한 공간 기반의 시각화한 기법을 제시한다. 이러한 시각화 서비스는 사용자에게 빅 데이터 기반 관광 빅 데이터 서비스를 제공하여 사용자 측면에서 데이터의 가시성을 제고하고 활용 가능 성을 높이는데 기여하였다. The spread of digital business has produced a number of information and data, and has increased the uses of big data in various areas. Big data now becomes one of the strategic factors in public organization, companies, civic groups etc. However the domestic tourism industry has been unfamiliar with big data business. Therefore the use of big data can boost the potential of the t

In [ ]:
for i in range(10):
    temp = browser.find_element_by_xpath("//div[@class='srchResultListW']/ul/li/")

In [471]:
# title = browser.find_elements_by_xpath("//div[@class='cont ml60']/p[@class='title']") #제목
writer = browser.find_elements_by_xpath("//div[@class='cont ml60']/p[@class='etc']/span[@class='writer']/a") #제목


In [473]:
len(writer)

54

In [472]:
for temp in writer:
    print(temp.text)

김경식
김진국
조완섭
류관희
Kyeongsik Kim
Jinkook Kim
Wan-Sup Cho
Kwan-Hee Yoo
유영석
박범준
조선화
이수안
김진호
최성수(Sung Soo Choi)
지수영(Su Young chi)
류관희(Kwan Hee Yoo)
김진국(Jin Kook Kim)
박주석
김승현
류호철
이준기
이장호
이준용
Park, Jooseok
Kim, Seunghyun
Ryu, Hocheol
Lee, Zoonky
Lee, Jangho
Lee, Junyong
김상완
함유근
Kim, Sang-Wan
Hahm, Yu-Kun
장원구
이경호
최영환
조완섭
이경희
최혜린
이승원
김영아
이종호
고홍
김현창
Choi, Hye Rin
Lee, Seung Won
Kim, YoungAh
Lee, Jong Ho
Koh, Hong
Kim, Hyeon Chang
이우기(Wookey Lee)
조치선
이난규
함유근


In [359]:
a = urlopen('https://www.naver.com/')
bs(a.read(), 'html.parser').find_all('h2')

[<h2 class="blind">뉴스스탠드</h2>,
 <h2 class="blind">주제별 캐스트</h2>,
 <h2 class="blind">Sign in</h2>,
 <h2 class="blind">타임스퀘어</h2>]

In [375]:
driver.find_element_by_xpath('//div')

<selenium.webdriver.remote.webelement.WebElement (session="b1deb70f14e97cb8802c0f554fb3547f", element="ecddac9d-a6b7-4190-a53c-63afca402f56")>

In [296]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
driver=webdriver.Chrome('C:/Users/DJK-Office/Desktop/webdriver/msedgedriver.exe', options=options)
driver.get('http://www.riss.kr/index.do#search')
driver.switch_to.window(driver.window_handles[1])
driver.close()
driver.switch_to.window(driver.window_handles[0])
keyword=input()
xpath='http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query='+keyword+'&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=&isFDetailSearch=&sflag=1&searchQuery='+keyword+'&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn='
driver.get(xpath)
append_list=[]
try :
    for i in range(2,13) :
        pageXpath=('/html/body/div[1]/div[2]/div[4]/div[2]/div/div[4]/a['+str(i)+']')
        
        driver.find_element_by_xpath(pageXpath).click()
        for j in range(1,11) :
            list=[]
         
            titleXpath=('/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(j)+']/div[2]/p[1]/a')
            driver.find_element_by_xpath(titleXpath).send_keys(Keys.CONTROL + '\n')
            driver.switch_to.window(driver.window_handles[1])
            list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[1]/div/p/a').text)
            list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[4]/div/p').text)
            list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[1]/h3').text)
            try :
                x=1
                while True :
                            
                    driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(x)+']/a').click()
                    list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(x)+']/div[3]/p').text)
                    x+=1
            except :
                pass
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            append_list.append(list)
    

except :
    pass


while True :
    
    try :
        for i in range(4,14) :
            pageXpath=('/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/a['+str(i)+']')
            driver.find_element_by_xpath(pageXpath).click()
            for j in range(1,11) :
                list=[]

                titleXpath=('/html/body/div[1]/div[2]/div[4]/div[2]/div/div[2]/div[2]/ul/li['+str(j)+']/div[2]/p[1]/a')
                driver.find_element_by_xpath(titleXpath).send_keys(Keys.CONTROL + '\n')

                driver.switch_to.window(driver.window_handles[1])
                list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[1]/div/p/a').text)
                list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[4]/div/p').text)
                list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[1]/h3').text)
                try :
                    x=1
                    while True :
                            
                        driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(x)+']/a').click()
                        list.append(driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(x)+']/div[3]/p').text)
                        x+=1
                except :
                    pass
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                append_list.append(list)
           
    except :
        break
print(append_list)


IndexError: list index out of range

In [282]:
titles = browser.find_element_by_css_selector('.title')

In [289]:
print([i for i in titles])

[]


In [272]:
print(soup.find(''))

<ul class="globalMenu">
<!-- 로그인 -->
<!-- <link type="text/css" rel="stylesheet" href="/commons/css/tooltip.css?20191223" /> -->
<script src="/commons/js/tooltip.js?20220324" type="text/javascript"></script><div id="dhtmltooltip"></div><img alt="" id="dhtmlpointer" src="/commons/images/tooltipArrow.gif"/>
<li class="login" id="LoginTopli0"><a href="javascript:void(0);" onclick="javascript:clickInsert('HOME&gt;TOP&gt;로그인', 'https://www.riss.kr/LoginRedirect.do', '', '');document.location.href='https://www.riss.kr/LoginRedirect.do?loginParameter=%3FisDetailSearch%3DN%26searchGubun%3Dtrue%26viewYn%3DOP%26query%3D%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%26query%3D%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%26queryText%3D%26iStartCount%3D0orderBy%3D%26mat_type%3D%26mat_subtype%3D%26fulltext_kind%3D%26t_gubun%3D%26learning_type%3D%26ccl_code%3D%26inside_outside%3D%26fric_yn%3D%26image_yn%3D%26gubun%3D%26kdc%3D%26ttsUseYn%3D%26l_sub_code%3D%26fsearchMethod%3Dsearch%26sflag%3D1%26isFDetailSearch%3DN%26pa

In [ ]:
from selenium import webdriver
from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
def get_reference(URL):
    driver_path = r'C:/Users/DJK-Office/Desktop/webdriver/msedgedriver.exe'
    driver = webdriver.Chrome(driver_path, options=options)  #edge driver chrome으로도 되네
    driver.get(URL)
    
    html = driver.page_source
    soup = bs(html, "lxml")
    
    title = soup.find("h3", "title")
    title_txt = title.get_text("", strip=True)

WebDriverException: Message: chrome not reachable
  (Session info: MicrosoftEdge=103.0.1264.62)
